In [13]:
import random
from transformers import BertTokenizerFast, EncoderDecoderModel
from googletrans import Translator

In [14]:
def bert_paraphrase(model, tokenizer, text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids)
    return(tokenizer.decode(output_ids[0], skip_special_tokens=True))

def k_hop_path(source, targets, k, chain=None, replacement=False):
    if chain:
        return [source] + chain + (random.choices(targets, k=k-len(chain)) if replacement else random.sample(targets, k=k-len(chain))) + [source]
    else:
        return [source] + (random.choices(targets, k=k) if replacement else random.sample(targets, k=k)) + [source]

def path_translate(translator, path, text):
    for i in range(len(path)-1):
        text = translator.translate(text, src=path[i], dest=path[i+1]).text
    return text

def k_hop_paraphrase(translator, source, targets, k, text, chain=None, replacement=False):
    return path_translate(translator, k_hop_path(source, targets, k, chain=chain, replacement=replacement), text)


In [15]:
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-cased")
model = EncoderDecoderModel.from_pretrained("ahmetbagci/bert2bert-turkish-paraphrase-generation")

translator = Translator()

Original: son model arabalar çevreye daha mı az zarar veriyor?
Bert: son model arabalar çevre için daha az zararlı mı?
K-Hop_3: En son model arabalar çevreye zarar verdi mi?
K-Hop_5: En son otomobillere çevreye zarar verdi mi?
K-Hop_7: Çevrenin en son otomobilleri hasar gördü mü?

Original: çürük raporu alırsam iş bulmada sıkıntı yaşar mıyım?
Bert: çürük raporu alırsam iş bulamaz mıyım?
K-Hop_3: Çürümüş raporu alırsam, bir iş bulmak zor mu?
K-Hop_5: Tembel bir rapor alırsam, bir iş bulmak zor mu?
K-Hop_7: Çürük ilişkiyi alırsam iş bulmak zor buldum?

Original: ingilizce yazma yeteneğimi geliştirmenin en iyi yolu nedir?
Bert: ingilizce yazma becerilerimi nasıl geliştirebilirim?
K-Hop_3: İngilizce yazma yeteneğimi geliştirmenin en iyi yolu nedir?
K-Hop_5: İngilizce yazma yeteneğimi geliştirmenin en iyi yolu nedir?
K-Hop_7: İngilizce yazımımı geliştirmenin en iyi yolu nedir?

Original: şimdiye kadarki en iyi şaka nedir? niye ya?
Bert: şimdiye kadar duyduğun en iyi şaka nedir?
K-Hop_3: Şim

In [ ]:

source = "tr"
targets = ["en", "de", "es", "sv", "ru", "hu", "it", "fr"]
chain = ["en"]
replacement = False

sentences = [
    "son model arabalar çevreye daha mı az zarar veriyor?",
    "çürük raporu alırsam iş bulmada sıkıntı yaşar mıyım?",
    "ingilizce yazma yeteneğimi geliştirmenin en iyi yolu nedir?",
    "şimdiye kadarki en iyi şaka nedir? niye ya?",
    "yüksek bir ıq'nun bazı olumsuz etkileri nelerdir?",
    "tüm zamanların en iyi 10 kitabı hangileridir?"
]

for sentence in sentences:
    bert = bert_paraphrase(model, tokenizer, sentence)
    k_hop_3 = k_hop_paraphrase(translator, source, targets, 3, sentence, chain=chain, replacement=replacement)
    k_hop_5 = k_hop_paraphrase(translator, source, targets, 5, sentence, chain=chain, replacement=replacement)
    k_hop_7 = k_hop_paraphrase(translator, source, targets, 7, sentence, chain=chain, replacement=replacement)
    print(f"Original: {sentence}\nBert: {bert}\nK-Hop_3: {k_hop_3}\nK-Hop_5: {k_hop_5}\nK-Hop_7: {k_hop_7}\n")